# Import libraries

In [1]:
!pip install datasets
!pip install evaluate rouge_score
!pip install accelerate

  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b4e614f219ee20624f78138f3a99ed9d53e547c28e311d8c6ea7db70d567676f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [2]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset, DatasetDict
import evaluate
import numpy as np
import pandas as pd
import accelerate

2024-05-29 06:59:39.936176: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 06:59:39.936293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 06:59:40.217624: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Model

In [3]:
model_path ="vinai/bartpho-word"
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer, model = model, return_tensors = "pt")

config.json:   0%|          | 0.00/897 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


# Dataset

In [4]:

# Read the CSV files into DataFrames
train_df = pd.read_csv('/kaggle/input/dataset/train_with_importance_sentences.csv')
valid_df = pd.read_csv('/kaggle/input/dataset/valid_with_importance_sentences.csv')

# Convert DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# Create a DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "valid": valid_dataset
})

# Verify the structure
print(dataset_dict)

# Optionally save the combined DatasetDict
dataset_dict.save_to_disk('/kaggle/working/dataset')


DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'Segmented_document', 'Segmented_summary', 'Dataset', 'importance_sentences'],
        num_rows: 8363
    })
    valid: Dataset({
        features: ['Document', 'Summary', 'Segmented_document', 'Segmented_summary', 'Dataset', 'importance_sentences'],
        num_rows: 2091
    })
})


Saving the dataset (0/1 shards):   0%|          | 0/8363 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2091 [00:00<?, ? examples/s]

In [5]:
dataset = DatasetDict.load_from_disk('/kaggle/working/dataset')
#data = load_dataset("Valleyy/final_nlp_data")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Document', 'Summary', 'Segmented_document', 'Segmented_summary', 'Dataset', 'importance_sentences'],
        num_rows: 8363
    })
    valid: Dataset({
        features: ['Document', 'Summary', 'Segmented_document', 'Segmented_summary', 'Dataset', 'importance_sentences'],
        num_rows: 2091
    })
})

In [7]:
def data_processing(dataset):
    #document = dataset["Segmented_document"]
    document = dataset["importance_sentences"]
    summary = dataset["Segmented_summary"]
    doc_tokenizer = tokenizer(document, max_length = 1024, truncation = True)
    sum_tokenizer = tokenizer(summary, max_length = 1024, truncation = True)
    input_ids = doc_tokenizer["input_ids"]
    attention_mask = doc_tokenizer["attention_mask"]
    labels = sum_tokenizer["input_ids"]
    dataset = DatasetDict({"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels})
    return dataset

In [8]:
dataset = dataset.map(data_processing, remove_columns=[ 'Document', 'Summary', 'Segmented_document','Segmented_summary', 'Dataset','importance_sentences'], batched = True, batch_size = 128)

Map:   0%|          | 0/8363 [00:00<?, ? examples/s]

Map:   0%|          | 0/2091 [00:00<?, ? examples/s]

# Training

In [9]:
metric = evaluate.load("rouge")
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions[predictions == -100] = 1
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.eos_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True,)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [10]:
args = Seq2SeqTrainingArguments(output_dir = "/kaggle/working/",
                                evaluation_strategy="epoch",
                                save_strategy="epoch",
                               per_device_train_batch_size=3,
                               per_device_eval_batch_size=3,
                               learning_rate=1e-4,
                               weight_decay=1e-2,
                               load_best_model_at_end =True,
                               predict_with_generate=True,
                               num_train_epochs=5,
                               logging_strategy="epoch",
                               generation_max_length=1024,
                                save_total_limit = 1,
                               fp16=True)

trainer = Seq2SeqTrainer(model=model,
                        args=args,
                        train_dataset=dataset["train"],
                        eval_dataset=dataset["valid"],
                        tokenizer=tokenizer,
                        compute_metrics=compute_metrics,
                        data_collator = data_collator)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
torch.cuda.empty_cache()
torch.cuda.memory_allocated()
PYTORCH_CUDA_ALLOC_CONF=expandable_segments=256

In [12]:
import wandb
wandb.login(key="0e8c4295aa5f2866f43774ab572a2533b50d39ca")

wandb.init(project="NLP-bartpho")

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ngocanhkk1809 (ngocanhkk180923). Use `wandb login --relogin` to force relogin
wandb: wandb version 0.17.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240529_070046-18c13p7s
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dutiful-dream-3
wandb: ⭐️ View project at https://wandb.ai/ngocanhkk180923/NLP-bartpho
wandb: 🚀 View run at https://wandb.ai/ngocanhkk180923/NLP-bartpho/runs/18c13p7s


In [13]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,2.608500,2.411755,0.562100,0.260800,0.367700,0.367800,33.288400
2,1.787300,2.376938,0.582400,0.280100,0.384900,0.384700,34.434700
3,1.071000,2.523676,0.581100,0.274400,0.380200,0.380100,30.786200
4,0.528500,2.696395,0.595300,0.285400,0.386200,0.386000,34.164500
5,0.229200,2.806094,0.596900,0.288400,0.390800,0.390700,33.097600


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
/opt/conda/lib/python3.1

TrainOutput(global_step=6970, training_loss=1.2449183547513898, metrics={'train_runtime': 8427.3616, 'train_samples_per_second': 4.962, 'train_steps_per_second': 0.827, 'total_flos': 2.536652270857421e+16, 'train_loss': 1.2449183547513898, 'epoch': 5.0})

In [14]:
import os

# Path to the directory to save the model
save_model_dir = '/kaggle/working/save_model'

# Check if the directory exists, if not, create it
if not os.path.exists(save_model_dir):
    os.makedirs(save_model_dir)
    print(f"The directory '{save_model_dir}' has been created.")
else:
    print(f"The directory '{save_model_dir}' already exists.")

The directory '/kaggle/working/save_model' has been created.


In [15]:
trainer.save_model("/kaggle/working/save_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
